In [ ]:
# GPUU Bugfix
import tensorflow
physical_devices = tensorflow.config.list_physical_devices('GPU')
tensorflow.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
# Custom Callback
import os
from tensorflow.keras.callbacks import Callback
from tensorflow.summary import create_file_writer, scalar, text
import tensorflow.keras.backend as TF
from tensorflow.keras.models import Sequential, save_model
class MyCallback(Callback):
    def __init__(self, run_base_dir):
        self.save_counter = 0
        self.least_loss = -1
        self.base_dir = run_base_dir
        os.makedirs(self.base_dir, exist_ok=True)
        os.makedirs(os.path.join(self.base_dir, "models"), exist_ok=True)
        os.makedirs(os.path.join(self.base_dir, "logs"), exist_ok=True)
        self.summary_writer = create_file_writer(os.path.join(self.base_dir, "logs"))
        self.summary_writer.set_as_default()
        self.iters_since_last_model_save = 0
    def on_epoch_end(self, epoch, logs=None):
        iter_no = TF.get_value(self.model.optimizer.iterations)
        loss = logs['loss']
        scalar("loss", data=loss, step = iter_no)
        text("sample", generate_sample("<SOA> the trump administration"), step=iter_no)
        self.iters_since_last_model_save += 1
        if self.least_loss<0 or loss<self.least_loss:
            self.least_loss = loss
            print("Loss decreased in iter {}".format(iter_no))
            if self.iters_since_last_model_save>0:
                print("Saving model at iteration {} with loss {}".format(iter_no, loss))
                save_model(self.model, os.path.join(self.base_dir, "models", "model-{0:.4f}.h5".format(loss)))
                self.iters_since_last_model_save = 0
    def apply_lr(self):
        TF.set_value(self.model.optimizer.lr, K.get_value(0.0001))